In [142]:
import numpy as np
from sklearn.utils import resample

from dtree import *
import pdb

In [61]:
a = [
    [1, 2, 1, 4, 5, 6, 7],
    [1, 2, 1, 1, 4, 8, 5],
    [1, 2, 3, 9, 5, 0, 7],
    [1, 2, 1, 4, 6, 1, 8],
    [2, 1, 1, 8, 6, 4, 3]
]

In [62]:
np.unique(a)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [63]:
np.unique(a , axis=0)

array([[1, 2, 1, 1, 4, 8, 5],
       [1, 2, 1, 4, 5, 6, 7],
       [1, 2, 1, 4, 6, 1, 8],
       [1, 2, 3, 9, 5, 0, 7],
       [2, 1, 1, 8, 6, 4, 3]])

In [143]:
max_features = 0.5
import pdb;pdb.set_trace()
np.random.choice(range(len(a[0])), size = int(len(a[0])*max_features), replace = False)

--Return--
None
> /var/folders/nd/_sqvjy6j76l4cp3jhkntppjm0000gn/T/ipykernel_33693/2060745576.py(2)<module>()
      1 max_features = 0.5
----> 2 import pdb;pdb.set_trace()
      3 np.random.choice(range(len(a[0])), size = int(len(a[0])*max_features), replace = False)



ipdb>  max_features


0.5


ipdb>  max_features + 0.5


1.0


ipdb>  q


In [133]:
X = np.array([[1., 0.], [2., 1.], [0., 0.], [4, 3], [7, 8], [9, 1]])
y = np.array([0, 1, 2, 5, 4, 2])

In [135]:
np.unique(y)

array([0, 1, 2, 4, 5])

In [134]:
Xidx = resample(X, replace = True)
Xidx

array([[9., 1.],
       [1., 0.],
       [9., 1.],
       [1., 0.],
       [2., 1.],
       [4., 3.]])

In [127]:
Xidx, yidx, idx = resample(X, y,range(len(X[:,0])), replace = True)
Xidx, yidx, idx

(array([[1., 0.],
        [4., 3.],
        [0., 0.],
        [4., 3.]]),
 array([0., 5., 2., 5.]),
 [0, 3, 2, 3])

In [145]:
np.array(list(set(range(len(X[:,0]))) - set(idx)))

array([1, 4, 5])

In [129]:
X[:,0], X[idx]

(array([1., 2., 0., 4.]),
 array([[1., 0.],
        [4., 3.],
        [0., 0.],
        [4., 3.]]))

In [73]:
resample(a, n_samples=5, replace=True, random_state=0)

[[2, 1, 1, 8, 6, 4, 3],
 [1, 2, 1, 4, 5, 6, 7],
 [1, 2, 1, 4, 6, 1, 8],
 [1, 2, 1, 4, 6, 1, 8],
 [1, 2, 1, 4, 6, 1, 8]]

In [149]:
np.zeros((X.shape[0],2))

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [150]:
counts = {i:0 for i in range(5)}

In [153]:
counts[3] = 4

In [154]:
counts

{0: 0, 1: 0, 2: 10, 3: 4, 4: 0}

In [158]:
sorted(counts, key= lambda x: counts[x], reverse=True)[0]

2

In [ ]:
def vectorize(V:dict, docwords:Sequence) -> np.ndarray:
    """
    Return a row vector (based upon V) for docwords with the word counts. 
    The first element of the returned vector is the count of unknown words. So |V| is |uniquewords|+1.
    """
    zarr = np.zeros(len(V))
    myarr = np.unique(docwords, return_counts=True)
    
    def func1(V, x):
        if x in V:
            return V[x] #np.where(np.array(list(V.keys())) == x)[0][0]
        else:
            return 0

    v_func1 = np.vectorize(func1)
    ind = v_func1(V, myarr[0])
    def func2(ind, myarr):
        zarr[ind] += myarr

    v_func2 = np.vectorize(func2, otypes = [object])
    v_func2(ind, myarr[1])
    return zarr

In [138]:
class RandomForest621:
    def __init__(self, n_estimators=10, oob_score=False):
        self.n_estimators = n_estimators
        self.oob_score = oob_score
        self.oob_score_ = np.nan
        #self.nunique = np.unique(y)

    def fit(self, X, y):
        """
        Given an (X, y) training set, fit all n_estimators trees to different,
        bootstrapped versions of the training data.  Keep track of the indexes of
        the OOB records for each tree.  After fitting all of the trees in the forest,
        compute the OOB validation score estimate and store as self.oob_score_, to
        mimic sklearn.
        """
        self.nunique = len(np.unique(y))
        trees = []
        oob_idx = []
        for i in range(self.n_estimators):
            Xb, yb, idx = resample(X, y, range(len(X[:,0])), replace=True)
            oob_idx.append(np.array(list(set(range(len(X[:,0]))) - set(idx))))
            ti = self.tname.fit(Xb, yb)
            trees.append(ti.root)
        self.trees = trees

        if self.oob_score:
            self.oob_score_ = '... compute OOB score ...'

In [139]:
class RandomForestRegressor621(RandomForest621):
    def __init__(self, n_estimators=10, min_samples_leaf=3, max_features=0.3, oob_score=False):
        super().__init__(n_estimators, oob_score=oob_score)
        self.trees = self.fit

    def predict(self, X_test) -> np.ndarray:
        """
        Given a 2D nxp array with one or more records, compute the weighted average
        prediction from all trees in this forest. Weight each trees prediction by
        the number of observations in the leaf making that prediction.  Return a 1D vector
        with the predictions for each input record of X_test.
        """
        y_pred=[]
        for x in X_test:
            leaves = set()
            njobs, ysum = 0, 0
            for i in self.trees:
                leaves.add(i.leaf(X_test)) #set of leaves reached by x
                nobs += len(i.leaf(X_test))
                ysum += i.leaf(X_test).y
            y_pred.append(ysum / nobs)

        return np.array(y_pred)
        
    def score(self, X_test, y_test) -> float:
        """
        Given a 2D nxp X_test array and 1D nx1 y_test array with one or more records,
        collect the prediction for each record and then compute R^2 on that and y_test.
        """
        return r2_score(y_test, self.predict(X_test))

In [136]:
d = {'a': 2, 'b': 4, 'c': 32}
max(d.values())

32

In [141]:
class RandomForestClassifier621(RandomForest621):
    def __init__(self, n_estimators=10, min_samples_leaf=3, max_features=0.3, oob_score=False):
        super().__init__(n_estimators, oob_score=oob_score)
        n_estimators = n_estimators
        self.min_samples_leaf = min_samples_leaf
        self.trees = self.fit

    def predict(self, X_test) -> np.ndarray:
        counts = {}
        y_pred=[]
        for x in X_test:
            for i in self.trees:
                xleaf = i.leaf(X_test) #set of leaves reached by x
                for j in xleaf.y:
                    counts[j] += 1
            y_pred.append(max(counts.values()))
        return np.array(y_pred)
        
    def score(self, X_test, y_test) -> float:
        """
        Given a 2D nxp X_test array and 1D nx1 y_test array with one or more records,
        collect the predicted class for each record and then compute accuracy between
        that and y_test.
        """
        return accuracy_score(y_test, self.predict(X_test))  